# A Minute of Your Time: Data Analysis

In [ ]:
import datetime
import dateutil
import json
import matplotlib.pyplot as plt

import pandas as pd

from scripts import text_helpers

## Load the data

In [ ]:
# Set this to the location of your data file
data_file_location = '../../data/pull-requests.json'

In [ ]:
# Parse the JSON file
def ensure_camel(s):
    """
    Convert a string to camel case.
    Some of the JSON properties in the response from the Azure DevOps API are not camel-cased.
    """
    allowed_names = ['_links']
    return s if text_helpers.iscamel(s) or s in allowed_names else text_helpers.camel(s)

with open(data_file_location, 'r', encoding='utf-8') as pull_requests_json_file:
    pull_requests_raw = json.load(pull_requests_json_file, object_hook=lambda d: text_helpers.remap_keys(ensure_camel, d))

In [ ]:
# Create a data frame of pull requests
def get_data_from_pull_request(pull_request):
    """
    Extract the information we want to process from a pull request API object.
    """
    return [
        pull_request['pullRequestId'],
        pull_request['createdBy']['displayName'],
        dateutil.parser.parse(pull_request['creationDate']),
        dateutil.parser.parse(pull_request['closedDate']),
        len(pull_request['reviewers'])
    ]

pull_requests = pd.DataFrame(
    [get_data_from_pull_request(pr) for pr in pull_requests_raw],
    columns=['id', 'author', 'created_time', 'merged_time', 'num_reviewers'])

In [ ]:
# Add a column for wall-clock time to complete
pull_requests['ttl'] = pull_requests['merged_time'] - pull_requests['created_time']
pull_requests['ttl']

In [ ]:
pull_requests.head()

In [ ]:
pull_requests.dtypes

## Exploratory data analysis

In [ ]:
# What range of data do we have?
first_merge = pull_requests['merged_time'].min()
last_merge = pull_requests['merged_time'].max()
print(f"Data goes from {first_merge.date()} to {last_merge.date()}.")

In [ ]:
# Breakdown of all PR completion times
pull_requests['ttl'].describe()

In [ ]:
# Breakdown of PRs completed in under an hour
pull_requests['ttl'][lambda x: x < datetime.timedelta(hours=1)].describe()

In [ ]:
# Breakdown of PRs completed in over 5 days
pull_requests['ttl'][lambda x: x > datetime.timedelta(days=5)].describe()

In [ ]:
# Breakdown by author
pull_requests.groupby('author')['ttl'].describe()

In [ ]:
# Who completed the most PRs?
ttl_by_author = pull_requests.groupby('author')['ttl']
ttl_by_author.size().nlargest(5)

In [ ]:
# Check out the CIX team ...
cix_team = ['Brian Cristante', 'Chris Sidi🦉', 'David Staheli', 'Hank Weber', 'Jason Prickett', 'Josh Gross', 'Kellie Jos 🐉', 'Leah Antkiewicz', 'Lucas Killgore', 'Madhuri Gummalla', 'PJ Quirk', 'Yang Cao (VSNC)']
cix_pull_requests = pull_requests.join(pd.DataFrame({'author': cix_team}).set_index('author'), on='author', how='inner')
cix_pull_requests.groupby('author')['ttl'].describe()

In [ ]:
# Histogram of completion time
plt.figure()
pull_requests['ttl'].apply(lambda x: x.total_seconds()).plot.hist(bins=100)

plt.xlabel('Time to complete PR (seconds)')
plt.xlim([0, datetime.timedelta(days=10).total_seconds()])
plt.show()